In [1]:
import os

os.environ['HF_HOME'] = '/cluster/work/eliashk/models/mistral/full_precision'

In [2]:
import os

os.environ['HF_HOME'] = '/cluster/work/eliashk/models/mistral/full_precision'

import torch
torch.cuda.empty_cache()

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

tiny_dataset = load_dataset('json', data_files='single_output_10_samples.jsonl')
device = 'cuda'
base_model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

In [4]:
# split tiny_dataset into train and eval 80/20
# tiny_dataset = tiny_dataset['train'].train_test_split(test_size=0.2, seed=42)
tiny_dataset['train']

Dataset({
    features: ['input', 'output'],
    num_rows: 10
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

def tokenize_function(examples):
    return tokenizer(examples['input'], examples['output'], padding="max_length", truncation=True)

In [6]:
tokenized_datasets = tiny_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
model = AutoModelForCausalLM.from_pretrained(base_model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="/cluster/work/eliashk/models/mistral/full_precision/test_trainer")

In [10]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="/cluster/work/eliashk/models/mistral/full_precision/test_trainer", evaluation_strategy="epoch")

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tiny_dataset,
    compute_metrics=compute_metrics,
)

/cluster/home/eliashk/falcon/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 190.12 MiB is free. Including non-PyTorch memory, this process has 15.71 GiB memory in use. Of the allocated memory 15.46 GiB is allocated by PyTorch, and 1.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)